In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [2]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from vaep.domain.vaep.vaep import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load VAEP Data

In [3]:
vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/data/vaep_values_v2.csv")
vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value
0,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.059166,0.004202,0.000000,-0.000000,0.000000
1,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.013482,0.020271,-0.045684,-0.016070,-0.061754
2,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.041042,0.016021,0.020770,-0.002539,0.018231
3,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.028680,0.018624,-0.012362,-0.002603,-0.014965
4,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.032836,0.015452,0.004157,0.003172,0.007328


In [4]:
vaep_data['Season'] = vaep_data['match_id'].apply(lambda x: int(x[:4]))
vaep_2021 = vaep_data[vaep_data['Season'] == 2021]
vaep_2022 = vaep_data[vaep_data['Season'] == 2022]
vaep_2023 = vaep_data[vaep_data['Season'] == 2023]

In [5]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats.csv")

In [6]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [7]:
vaep_data['vaep_value'].max(), vaep_data['vaep_value'].min()

(1.220352672064246, -0.8738073321012418)

Match Level

In [8]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [9]:
match_id = "2022F4_Geelong_Sydney"

In [10]:
match_vaep_ranking = get_match_player_level_vaep(vaep_data, match_id)
match_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
13,Isaac Smith,Geelong,2.822755,2.950410,-0.127655,69,1.0,1.0,37.0,1.000000,1.000000,0.254465,2.801788,2.923825,-0.625088
5,Chad Warner,Sydney,2.381427,2.438741,-0.057314,69,2.0,2.0,30.0,0.871533,0.857927,0.328517,2.228740,2.272627,-0.310070
42,Tyson Stengle,Geelong,2.205213,2.362627,-0.157414,34,3.0,3.0,39.0,0.820239,0.836793,0.223136,1.999934,2.175757,-0.758361
0,Brad Close,Geelong,2.201508,2.179804,0.021704,44,4.0,4.0,19.0,0.819161,0.786029,0.411704,1.995122,1.943079,0.043811
30,Patrick Dangerfield,Geelong,2.046373,2.123575,-0.077203,58,5.0,5.0,31.0,0.774002,0.770417,0.307579,1.793685,1.871518,-0.399140


In [11]:
match_noshot_vaep_ranking = get_match_player_level_vaep(vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('vaep_value', ascending = False).head()

,player,team,vaep_value,offensive_value,defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_vaep_value,normalised_offensive_value,normalised_defensive_value,z_vaep_value,z_offensive_value,z_defensive_value
30,Patrick Dangerfield,Geelong,2.331390,2.408838,-0.077448,57,1.0,1.0,35.0,1.000000,1.000000,0.307321,3.270673,3.594630,-0.476898
24,Mark Blicavs,Geelong,2.018390,1.712078,0.306313,47,2.0,2.0,6.0,0.889978,0.771925,0.711327,2.710285,2.298008,1.269051
5,Chad Warner,Sydney,1.622908,1.569231,0.053677,66,3.0,3.0,16.0,0.750963,0.725166,0.445363,2.002223,2.032180,0.119663
0,Brad Close,Geelong,1.460659,1.376492,0.084167,42,4.0,5.0,13.0,0.693932,0.662075,0.477462,1.711736,1.673505,0.258382
13,Isaac Smith,Geelong,1.356541,1.408286,-0.051745,65,5.0,4.0,32.0,0.657333,0.672482,0.334380,1.525327,1.732673,-0.359961


In [12]:
match_stats = player_stats[player_stats['Match_ID'] == match_id]
match_stats[['Team', 'Player', "Goals", "Behinds", "Disposals"]]

,Team,Player,Goals,Behinds,Disposals
18967,Geelong,Brad Close,2,0,18
18968,Geelong,Brandan Parfitt,1,0,8
18969,Geelong,Cameron Guthrie,1,1,16
18970,Geelong,Gary Rohan,0,1,7
18971,Geelong,Gryan Miers,0,1,18
18972,Geelong,Isaac Smith,3,1,32
18973,Geelong,Jack Henry,0,0,10
18974,Geelong,Jake Kolodjashnij,0,0,17
18975,Geelong,Jed Bews,0,0,6
18976,Geelong,Jeremy Cameron,2,1,18


Player Ratings

In [13]:
players_vaep = get_player_vaep(vaep_data)
players_vaep.sort_values('vaep_value', ascending = False).head(5)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,104.733358,105.591735,-0.858377,4145
817,Taylor Walker,Adelaide,95.243477,100.930268,-5.686791,1592
853,Tom Hawkins,Geelong,86.141119,91.125648,-4.984530,1855
138,Charlie Cameron,Brisbane Lions,86.100206,91.505459,-5.405253,1678
149,Christian Petracca,Melbourne,82.787536,83.373646,-0.586109,4317


In [14]:
players_noshot_vaep = get_player_vaep(vaep_data[vaep_data['action_type'] != "Shot"])
players_noshot_vaep.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,93.701333,93.393606,0.307727,3995
791,Shai Bolton,Richmond,86.413406,88.023517,-1.610111,2638
149,Christian Petracca,Melbourne,84.910538,84.176928,0.733610,4100
5,Aaron Naughton,Western Bulldogs,80.580236,82.871684,-2.291448,1467
319,Jack Macrae,Western Bulldogs,73.554911,70.556540,2.998372,4538
514,Lachie Neale,Brisbane Lions,71.376079,68.400504,2.975575,3879
858,Tom Liberatore,Western Bulldogs,70.864829,65.400055,5.464774,3593
138,Charlie Cameron,Brisbane Lions,70.825081,72.898457,-2.073377,1401
264,Harris Andrews,Brisbane Lions,69.338595,25.594264,43.744332,3084
198,Darcy Parish,Essendon,68.625374,69.404015,-0.778642,3681


Normalise for minutes played

In [15]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
817,Taylor Walker,Adelaide,95.243477,100.930268,-5.686791,1592,4408.0,52.0,2.160696,2.289707,-0.129011
142,Charlie Curnow,Carlton,80.929306,86.215738,-5.286432,1378,4112.0,45.0,1.968125,2.096686,-0.128561
584,Marcus Bontempelli,Western Bulldogs,104.733358,105.591735,-0.858377,4145,5750.0,68.0,1.821450,1.836378,-0.014928
350,Jacob van Rooyen,Melbourne,18.956530,19.324127,-0.367597,382,1105.0,15.0,1.715523,1.748790,-0.033267
856,Tom J. Lynch,Richmond,54.375741,58.071594,-3.695853,966,3282.0,37.0,1.656787,1.769397,-0.112610
198,Darcy Parish,Essendon,69.531777,70.581357,-1.049580,3723,4298.0,53.0,1.617771,1.642191,-0.024420
834,Toby Greene,Greater Western Sydney,70.326250,74.540628,-4.214378,1895,4480.0,49.0,1.569782,1.663853,-0.094071
791,Shai Bolton,Richmond,80.817509,84.223939,-3.406430,2857,5258.0,62.0,1.537039,1.601825,-0.064786
52,Bayley Fritsch,Melbourne,79.340130,84.540334,-5.200204,1410,5212.0,62.0,1.522259,1.622033,-0.099774
423,Jeremy Cameron,Geelong,76.533429,82.580833,-6.047405,1922,5028.0,56.0,1.522145,1.642419,-0.120275


In [16]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
791,Shai Bolton,Richmond,86.413406,88.023517,-1.610111,2638,5258.0,62.0,1.643465,1.674087,-0.030622
584,Marcus Bontempelli,Western Bulldogs,93.701333,93.393606,0.307727,3995,5750.0,68.0,1.629588,1.624237,0.005352
198,Darcy Parish,Essendon,68.625374,69.404015,-0.778642,3681,4298.0,53.0,1.596682,1.614798,-0.018116
239,Elliot Yeo,West Coast,25.681291,22.810296,2.870994,1165,1661.0,24.0,1.546134,1.373287,0.172847
702,Patrick Dangerfield,Geelong,51.340176,52.345500,-1.005325,2319,3342.0,48.0,1.536211,1.566293,-0.030082
856,Tom J. Lynch,Richmond,49.784530,51.897380,-2.112850,792,3282.0,37.0,1.516896,1.581273,-0.064377
149,Christian Petracca,Melbourne,84.910538,84.176928,0.733610,4100,5937.0,68.0,1.430193,1.417836,0.012357
142,Charlie Curnow,Carlton,58.566117,60.819098,-2.252982,1130,4112.0,45.0,1.424273,1.479064,-0.054790
5,Aaron Naughton,Western Bulldogs,80.580236,82.871684,-2.291448,1467,5677.0,67.0,1.419416,1.459780,-0.040364
514,Lachie Neale,Brisbane Lions,71.376079,68.400504,2.975575,3879,5185.0,60.0,1.376588,1.319200,0.057388


In [17]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
817,Taylor Walker,Adelaide,95.243477,100.930268,-5.686791,1592,4408.0,52.0,2.160696,2.289707,-0.129011
142,Charlie Curnow,Carlton,80.929306,86.215738,-5.286432,1378,4112.0,45.0,1.968125,2.096686,-0.128561
584,Marcus Bontempelli,Western Bulldogs,104.733358,105.591735,-0.858377,4145,5750.0,68.0,1.821450,1.836378,-0.014928
856,Tom J. Lynch,Richmond,54.375741,58.071594,-3.695853,966,3282.0,37.0,1.656787,1.769397,-0.112610
350,Jacob van Rooyen,Melbourne,18.956530,19.324127,-0.367597,382,1105.0,15.0,1.715523,1.748790,-0.033267
834,Toby Greene,Greater Western Sydney,70.326250,74.540628,-4.214378,1895,4480.0,49.0,1.569782,1.663853,-0.094071
423,Jeremy Cameron,Geelong,76.533429,82.580833,-6.047405,1922,5028.0,56.0,1.522145,1.642419,-0.120275
198,Darcy Parish,Essendon,69.531777,70.581357,-1.049580,3723,4298.0,53.0,1.617771,1.642191,-0.024420
52,Bayley Fritsch,Melbourne,79.340130,84.540334,-5.200204,1410,5212.0,62.0,1.522259,1.622033,-0.099774
138,Charlie Cameron,Brisbane Lions,86.100206,91.505459,-5.405253,1678,5691.0,67.0,1.512919,1.607898,-0.094979


In [18]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
791,Shai Bolton,Richmond,86.413406,88.023517,-1.610111,2638,5258.0,62.0,1.643465,1.674087,-0.030622
584,Marcus Bontempelli,Western Bulldogs,93.701333,93.393606,0.307727,3995,5750.0,68.0,1.629588,1.624237,0.005352
198,Darcy Parish,Essendon,68.625374,69.404015,-0.778642,3681,4298.0,53.0,1.596682,1.614798,-0.018116
856,Tom J. Lynch,Richmond,49.784530,51.897380,-2.112850,792,3282.0,37.0,1.516896,1.581273,-0.064377
702,Patrick Dangerfield,Geelong,51.340176,52.345500,-1.005325,2319,3342.0,48.0,1.536211,1.566293,-0.030082
142,Charlie Curnow,Carlton,58.566117,60.819098,-2.252982,1130,4112.0,45.0,1.424273,1.479064,-0.054790
5,Aaron Naughton,Western Bulldogs,80.580236,82.871684,-2.291448,1467,5677.0,67.0,1.419416,1.459780,-0.040364
149,Christian Petracca,Melbourne,84.910538,84.176928,0.733610,4100,5937.0,68.0,1.430193,1.417836,0.012357
873,Touk Miller,Gold Coast,58.758809,62.375718,-3.616908,3362,4492.0,53.0,1.308077,1.388596,-0.080519
429,Jesse Hogan,Greater Western Sydney,46.230822,48.886226,-2.655403,1145,3548.0,42.0,1.303011,1.377853,-0.074842


In [19]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
427,Jeremy McGovern,West Coast,28.495187,11.370109,17.125077,1530,2293.0,30.0,1.242703,0.495862,0.746842
197,Darcy Moore,Collingwood,64.866674,25.905868,38.960806,2646,5218.0,55.0,1.243133,0.496471,0.746662
779,Sam Taylor,Greater Western Sydney,49.895569,13.728629,36.166940,2486,4916.0,52.0,1.014963,0.279264,0.735699
449,Jordan Ridley,Essendon,58.105907,21.812744,36.293163,3023,5201.0,59.0,1.117206,0.419395,0.697811
264,Harris Andrews,Brisbane Lions,69.856550,26.177450,43.679100,3088,6317.0,65.0,1.105850,0.414397,0.691453
480,Josh Worrell,Adelaide,12.282584,5.338930,6.943654,644,1037.0,13.0,1.184434,0.514844,0.669591
806,Steven May,Melbourne,61.684141,24.429402,37.254738,3139,5855.0,62.0,1.053529,0.417240,0.636289
100,Brennan Cox,Fremantle,47.604233,19.341693,28.262540,2420,4527.0,51.0,1.051562,0.427252,0.624311
845,Tom Clurey,Port Adelaide,19.953517,6.459267,13.494250,810,2210.0,25.0,0.902874,0.292275,0.610600
389,James Sicily,Hawthorn,44.035751,23.139930,20.895821,2615,3428.0,36.0,1.284590,0.675027,0.609563


In [20]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
197,Darcy Moore,Collingwood,64.112634,25.027691,39.084943,2637,5218.0,55.0,1.228682,0.479641,0.749041
427,Jeremy McGovern,West Coast,27.977955,10.834388,17.143567,1528,2293.0,30.0,1.220146,0.472498,0.747648
779,Sam Taylor,Greater Western Sydney,49.895569,13.728629,36.166940,2486,4916.0,52.0,1.014963,0.279264,0.735699
449,Jordan Ridley,Essendon,58.076449,21.738751,36.337698,3019,5201.0,59.0,1.116640,0.417973,0.698668
264,Harris Andrews,Brisbane Lions,69.338595,25.594264,43.744332,3084,6317.0,65.0,1.097651,0.405165,0.692486
480,Josh Worrell,Adelaide,12.282584,5.338930,6.943654,644,1037.0,13.0,1.184434,0.514844,0.669591
806,Steven May,Melbourne,63.110857,25.900205,37.210652,3133,5855.0,62.0,1.077897,0.442360,0.635536
100,Brennan Cox,Fremantle,46.458829,18.165846,28.292983,2414,4527.0,51.0,1.026261,0.401278,0.624983
389,James Sicily,Hawthorn,44.172283,23.204279,20.968004,2604,3428.0,36.0,1.288573,0.676904,0.611669
845,Tom Clurey,Port Adelaide,19.953517,6.459267,13.494250,810,2210.0,25.0,0.902874,0.292275,0.610600


VAEP by Action

In [21]:
action_vaep = get_vaep_action_summary(vaep_data)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-343.917625,-217.762499,-126.155126,3238,0.003137,-0.106213,-0.067252,-0.038961,-0.019042,-0.013191,-0.081291
12,Tackle,-3.600396,1.131570,-4.731965,314,0.000304,-0.011466,0.003604,-0.015070,-0.000199,0.000069,-0.003049
5,Handball,-1567.474936,-721.063541,-846.411395,181764,0.176102,-0.008624,-0.003967,-0.004657,-0.086789,-0.043677,-0.545406
7,Kick,877.467427,1834.705645,-957.238218,239630,0.232165,0.003662,0.007656,-0.003995,0.048584,0.111134,-0.616820
0,Carry,2528.036372,2441.014206,87.022166,163299,0.158212,0.015481,0.014948,0.000533,0.139974,0.147861,0.056075
11,Spoil,777.831695,-96.544387,874.376082,41219,0.039935,0.018871,-0.002342,0.021213,0.043067,-0.005848,0.563425
4,Gather,862.522735,783.105773,79.416962,40992,0.039715,0.021041,0.019104,0.001937,0.047757,0.047435,0.051174
6,Hard Ball Get,1221.226743,757.472123,463.754620,52402,0.050770,0.023305,0.014455,0.008850,0.067618,0.045883,0.298831
13,Uncontested Mark,3699.940182,3599.901300,100.038882,114870,0.111292,0.032210,0.031339,0.000871,0.204860,0.218058,0.064462
9,Loose Ball Get,3752.658322,2773.639038,979.019284,110464,0.107023,0.033972,0.025109,0.008863,0.207779,0.168009,0.630855


In [22]:
match_data = get_match(vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-0.189218,-0.138525,-0.050693,3,0.001858,-0.063073,-0.046175,-0.016898,-0.006186,-0.004611,-0.092437
5,Handball,-2.311243,-1.595939,-0.715304,280,0.173375,-0.008254,-0.005700,-0.002555,-0.075558,-0.053126,-1.304332
7,Kick,2.874659,3.779315,-0.904657,374,0.231579,0.007686,0.010105,-0.002419,0.093977,0.125807,-1.649610
11,Spoil,0.503647,-0.278438,0.782085,58,0.035913,0.008684,-0.004801,0.013484,0.016465,-0.009269,1.426104
0,Carry,2.491921,2.937242,-0.445320,265,0.164087,0.009403,0.011084,-0.001680,0.081464,0.097776,-0.812026
4,Gather,0.557927,0.608978,-0.051051,59,0.036533,0.009456,0.010322,-0.000865,0.018239,0.020272,-0.093090
6,Hard Ball Get,1.579538,1.579406,0.000132,80,0.049536,0.019744,0.019743,0.000002,0.051637,0.052576,0.000240
12,Uncontested Mark,5.232502,5.424484,-0.191982,193,0.119505,0.027111,0.028106,-0.000995,0.171058,0.180571,-0.350073
9,Loose Ball Get,5.340255,3.912918,1.427337,190,0.117647,0.028107,0.020594,0.007512,0.174580,0.130254,2.602699
1,Contested Mark,1.502921,1.306433,0.196487,16,0.009907,0.093933,0.081652,0.012280,0.049133,0.043489,0.358288


In [23]:
player = "Christian Petracca"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-6.040536,-3.493575,-2.546961,959,0.222145,-0.006299,-0.003643,-0.002656,-0.072964,-0.041903,4.345540
2,Error,-2.632014,-2.041990,-0.590024,26,0.006023,-0.101231,-0.078538,-0.022693,-0.031792,-0.024492,1.006680
10,Shot,-2.123001,-0.803282,-1.319719,217,0.050266,-0.009783,-0.003702,-0.006082,-0.025644,-0.009635,2.251661
11,Spoil,0.163174,0.129818,0.033356,18,0.004170,0.009065,0.007212,0.001853,0.001971,0.001557,-0.056911
8,Knock On,0.390676,0.251164,0.139513,12,0.002780,0.032556,0.020930,0.011626,0.004719,0.003013,-0.238032
1,Contested Mark,1.738603,1.729719,0.008884,19,0.004401,0.091505,0.091038,0.000468,0.021001,0.020747,-0.015158
7,Kick,6.707065,9.404151,-2.697086,870,0.201529,0.007709,0.010809,-0.003100,0.081015,0.112795,4.601677
3,Free For,7.142170,6.372957,0.769213,59,0.013667,0.121054,0.108016,0.013038,0.086271,0.076439,-1.312406
6,Hard Ball Get,8.473341,6.999874,1.473467,319,0.073894,0.026562,0.021943,0.004619,0.102350,0.083958,-2.513980
4,Gather,14.017348,11.831174,2.186173,336,0.077832,0.041718,0.035212,0.006506,0.169317,0.141905,-3.729976


In [24]:
player = "Nick Daicos"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-7.138480,-2.612254,-4.526225,583,0.212154,-0.012244,-0.004481,-0.007764,-0.183661,-0.067141,115.375739
1,Error,-1.074047,-0.305477,-0.768570,9,0.003275,-0.119339,-0.033942,-0.085397,-0.027633,-0.007851,19.591226
10,Spoil,-0.157284,-0.092682,-0.064602,8,0.002911,-0.019661,-0.011585,-0.008075,-0.004047,-0.002382,1.646735
7,Knock On,0.201100,0.135647,0.065453,6,0.002183,0.033517,0.022608,0.010909,0.005174,0.003486,-1.668427
5,Hard Ball Get,1.193296,0.698291,0.495006,77,0.028020,0.015497,0.009069,0.006429,0.030702,0.017948,-12.617940
3,Gather,2.488562,1.943433,0.545129,103,0.037482,0.024161,0.018868,0.005293,0.064027,0.049951,-13.895608
9,Shot,2.775056,3.413876,-0.638820,56,0.020378,0.049555,0.060962,-0.011407,0.071398,0.087745,16.283834
6,Kick,4.577427,6.920644,-2.343217,630,0.229258,0.007266,0.010985,-0.003719,0.117770,0.177877,59.729768
2,Free For,5.181880,3.765238,1.416643,52,0.018923,0.099652,0.072408,0.027243,0.133321,0.096776,-36.110926
11,Uncontested Mark,5.221378,5.179405,0.041973,194,0.070597,0.026914,0.026698,0.000216,0.134337,0.133123,-1.069908


In [25]:
player = "Darcy Moore"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-0.908128,-0.028313,-0.879814,6,0.002268,-0.151355,-0.004719,-0.146636,-0.014000,-0.001093,-0.022582
5,Handball,-0.221515,0.904555,-1.126070,294,0.111111,-0.000753,0.003077,-0.003830,-0.003415,0.034917,-0.028903
4,Gather,0.274387,0.461907,-0.187520,48,0.018141,0.005716,0.009623,-0.003907,0.004230,0.017830,-0.004813
7,Kick,0.558218,5.436419,-4.878201,616,0.232804,0.000906,0.008825,-0.007919,0.008606,0.209853,-0.125208
10,Shot,0.754039,0.878177,-0.124137,9,0.003401,0.083782,0.097575,-0.013793,0.011624,0.033899,-0.003186
8,Knock On,1.898841,0.698978,1.199863,59,0.022298,0.032184,0.011847,0.020337,0.029273,0.026981,0.030797
6,Hard Ball Get,2.612956,0.515696,2.097260,104,0.039305,0.025125,0.004959,0.020166,0.040282,0.019907,0.053830
3,Free For,4.975353,2.535444,2.439909,42,0.015873,0.118461,0.060368,0.058093,0.076701,0.097871,0.062625
9,Loose Ball Get,5.547598,1.459742,4.087856,209,0.078987,0.026544,0.006984,0.019559,0.085523,0.056348,0.104922
0,Carry,6.529754,1.490075,5.039679,309,0.116780,0.021132,0.004822,0.016310,0.100664,0.057519,0.129353


In [26]:
player = "Taylor Walker"
player_data = get_player(vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value").round(2)

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-5.49,-5.69,0.19,212,0.13,-0.03,-0.03,0.00,-0.06,-0.06,-0.03
11,Spoil,-1.45,-1.11,-0.34,36,0.02,-0.04,-0.03,-0.01,-0.02,-0.01,0.06
2,Error,-0.86,-0.74,-0.12,7,0.00,-0.12,-0.11,-0.02,-0.01,-0.01,0.02
0,Carry,-0.59,-0.23,-0.36,124,0.08,-0.00,-0.00,-0.00,-0.01,-0.00,0.06
7,Kick,1.61,2.11,-0.49,243,0.15,0.01,0.01,-0.00,0.02,0.02,0.09
4,Gather,1.64,1.67,-0.03,45,0.03,0.04,0.04,-0.00,0.02,0.02,0.01
8,Knock On,1.85,1.83,0.03,20,0.01,0.09,0.09,0.00,0.02,0.02,-0.00
6,Hard Ball Get,2.16,2.25,-0.08,114,0.07,0.02,0.02,-0.00,0.02,0.02,0.01
9,Loose Ball Get,10.30,10.10,0.20,127,0.08,0.08,0.08,0.00,0.11,0.10,-0.04
3,Free For,12.48,12.49,-0.01,63,0.04,0.20,0.20,-0.00,0.13,0.12,0.00


By Season

In [27]:
players_2021 = get_player_vaep(vaep_2021)
players_2022 = get_player_vaep(vaep_2022)
players_2023 = get_player_vaep(vaep_2023)

In [28]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [29]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [30]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [31]:
player_ratings_2021.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
420,Marcus Bontempelli,Western Bulldogs,36.626818,37.545180,-0.918362,1569,2193,26,1.670170,1.712047,-0.041877
232,Jack Macrae,Western Bulldogs,36.079795,35.876386,0.203410,1872,2154,26,1.675014,1.665570,0.009443
145,Darcy Parish,Essendon,33.617026,33.467565,0.149461,1492,1890,23,1.778679,1.770771,0.007908
110,Christian Petracca,Melbourne,32.981547,34.062685,-1.081138,1524,2237,25,1.474365,1.522695,-0.048330
268,Jake Stringer,Essendon,32.445915,33.274495,-0.828579,661,1526,19,2.126207,2.180504,-0.054297
100,Charlie Cameron,Brisbane Lions,30.892078,32.779424,-1.887346,560,2082,24,1.483769,1.574420,-0.090651
112,Clayton Oliver,Melbourne,30.606756,30.599565,0.007190,1702,2211,25,1.384295,1.383969,0.000325
37,Bayley Fritsch,Melbourne,30.534083,32.530749,-1.996667,523,2049,24,1.490194,1.587640,-0.097446
618,Tom Liberatore,Western Bulldogs,29.448682,28.294665,1.154017,1284,1967,25,1.497137,1.438468,0.058669
194,Harris Andrews,Brisbane Lions,28.510448,9.530053,18.980394,1109,2224,23,1.281945,0.428510,0.853435


In [32]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,32.445915,33.274495,-0.828579,661,1526,19,2.126207,2.180504,-0.054297
305,Jeremy Cameron,Geelong,23.169250,24.483142,-1.313891,386,1237,15,1.873019,1.979235,-0.106216
589,Taylor Walker,Adelaide,24.923991,26.442928,-1.518938,441,1369,16,1.820598,1.931551,-0.110952
145,Darcy Parish,Essendon,33.617026,33.467565,0.149461,1492,1890,23,1.778679,1.770771,0.007908
232,Jack Macrae,Western Bulldogs,36.079795,35.876386,0.203410,1872,2154,26,1.675014,1.665570,0.009443
420,Marcus Bontempelli,Western Bulldogs,36.626818,37.545180,-0.918362,1569,2193,26,1.670170,1.712047,-0.041877
200,Harry McKay,Carlton,26.334882,28.825285,-2.490403,405,1637,19,1.608728,1.760860,-0.152132
381,Lance Franklin,Sydney,25.514318,27.278500,-1.764182,459,1615,18,1.579834,1.689071,-0.109237
662,Zac Williams,Carlton,15.407802,12.088601,3.319201,621,1029,14,1.497357,1.174791,0.322566
42,Ben Cunnington,North Melbourne,18.521299,17.247052,1.274247,880,1237,15,1.497276,1.394265,0.103011


In [33]:
player_ratings_2021.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
420,Marcus Bontempelli,Western Bulldogs,36.626818,37.545180,-0.918362,1569,2193,26,1.670170,1.712047,-0.041877
232,Jack Macrae,Western Bulldogs,36.079795,35.876386,0.203410,1872,2154,26,1.675014,1.665570,0.009443
110,Christian Petracca,Melbourne,32.981547,34.062685,-1.081138,1524,2237,25,1.474365,1.522695,-0.048330
145,Darcy Parish,Essendon,33.617026,33.467565,0.149461,1492,1890,23,1.778679,1.770771,0.007908
268,Jake Stringer,Essendon,32.445915,33.274495,-0.828579,661,1526,19,2.126207,2.180504,-0.054297
100,Charlie Cameron,Brisbane Lions,30.892078,32.779424,-1.887346,560,2082,24,1.483769,1.574420,-0.090651
37,Bayley Fritsch,Melbourne,30.534083,32.530749,-1.996667,523,2049,24,1.490194,1.587640,-0.097446
112,Clayton Oliver,Melbourne,30.606756,30.599565,0.007190,1702,2211,25,1.384295,1.383969,0.000325
104,Charlie Dixon,Port Adelaide,27.842837,29.352580,-1.509743,675,2276,24,1.223323,1.289656,-0.066333
200,Harry McKay,Carlton,26.334882,28.825285,-2.490403,405,1637,19,1.608728,1.760860,-0.152132


In [34]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,32.445915,33.274495,-0.828579,661,1526,19,2.126207,2.180504,-0.054297
305,Jeremy Cameron,Geelong,23.169250,24.483142,-1.313891,386,1237,15,1.873019,1.979235,-0.106216
589,Taylor Walker,Adelaide,24.923991,26.442928,-1.518938,441,1369,16,1.820598,1.931551,-0.110952
145,Darcy Parish,Essendon,33.617026,33.467565,0.149461,1492,1890,23,1.778679,1.770771,0.007908
200,Harry McKay,Carlton,26.334882,28.825285,-2.490403,405,1637,19,1.608728,1.760860,-0.152132
420,Marcus Bontempelli,Western Bulldogs,36.626818,37.545180,-0.918362,1569,2193,26,1.670170,1.712047,-0.041877
381,Lance Franklin,Sydney,25.514318,27.278500,-1.764182,459,1615,18,1.579834,1.689071,-0.109237
232,Jack Macrae,Western Bulldogs,36.079795,35.876386,0.203410,1872,2154,26,1.675014,1.665570,0.009443
37,Bayley Fritsch,Melbourne,30.534083,32.530749,-1.996667,523,2049,24,1.490194,1.587640,-0.097446
100,Charlie Cameron,Brisbane Lions,30.892078,32.779424,-1.887346,560,2082,24,1.483769,1.574420,-0.090651


In [35]:
player_ratings_2021.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,28.510448,9.530053,18.980394,1109,2224,23,1.281945,0.428510,0.853435
324,Jordan Ridley,Essendon,22.227210,7.689899,14.537310,1183,2008,22,1.106933,0.382963,0.723970
18,Aliir Aliir,Port Adelaide,24.902001,10.703666,14.198335,1042,2315,24,1.075680,0.462361,0.613319
163,Dougal Howard,St Kilda,19.611937,5.421788,14.190149,935,1983,21,0.989003,0.273413,0.715590
617,Tom Jonas,Port Adelaide,20.318982,6.283496,14.035487,985,2297,24,0.884588,0.273552,0.611036
253,Jacob Weitering,Carlton,22.603695,9.271938,13.331757,1063,2110,22,1.071265,0.439428,0.631837
580,Steven May,Melbourne,19.958586,8.142941,11.815645,1052,2145,23,0.930470,0.379624,0.550846
557,Sam Taylor,Greater Western Sydney,16.770933,4.967824,11.803109,829,1781,19,0.941658,0.278935,0.662724
391,Liam Jones,Carlton,18.360661,6.673404,11.687256,716,1814,19,1.012164,0.367883,0.644281
13,Alex Keath,Western Bulldogs,17.531742,5.958458,11.573284,803,2173,23,0.806799,0.274204,0.532595


In [36]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,28.510448,9.530053,18.980394,1109,2224,23,1.281945,0.428510,0.853435
308,Jeremy McGovern,West Coast,14.194930,4.930921,9.264009,672,1204,15,1.178981,0.409545,0.769436
324,Jordan Ridley,Essendon,22.227210,7.689899,14.537310,1183,2008,22,1.106933,0.382963,0.723970
163,Dougal Howard,St Kilda,19.611937,5.421788,14.190149,935,1983,21,0.989003,0.273413,0.715590
557,Sam Taylor,Greater Western Sydney,16.770933,4.967824,11.803109,829,1781,19,0.941658,0.278935,0.662724
73,Brennan Cox,Fremantle,10.537285,3.497551,7.039734,459,1085,12,0.971178,0.322355,0.648823
391,Liam Jones,Carlton,18.360661,6.673404,11.687256,716,1814,19,1.012164,0.367883,0.644281
540,Sam Collins,Gold Coast,16.684281,5.298782,11.385499,725,1788,20,0.933125,0.296352,0.636773
253,Jacob Weitering,Carlton,22.603695,9.271938,13.331757,1063,2110,22,1.071265,0.439428,0.631837
18,Aliir Aliir,Port Adelaide,24.902001,10.703666,14.198335,1042,2315,24,1.075680,0.462361,0.613319


2022

In [37]:
player_ratings_2022.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,36.980404,39.051748,-2.071344,514,1601,19,2.309832,2.439210,-0.129378
311,Jeremy Cameron,Geelong,34.581821,37.182107,-2.600286,833,2214,24,1.561961,1.679409,-0.117447
104,Charlie Curnow,Carlton,34.080586,36.425688,-2.345102,574,2022,22,1.685489,1.801468,-0.115979
211,Isaac Heeney,Sydney,32.448135,33.466696,-1.018562,896,2136,25,1.519107,1.566793,-0.047685
623,Tom Hawkins,Geelong,31.616207,33.756631,-2.140424,696,2366,25,1.336272,1.426738,-0.090466
38,Bayley Fritsch,Melbourne,31.053790,32.931280,-1.877489,514,2009,24,1.545734,1.639188,-0.093454
521,Peter Wright,Essendon,30.675617,33.417901,-2.742284,586,2162,22,1.418854,1.545694,-0.126840
514,Patrick Cripps,Carlton,30.188070,31.022549,-0.834478,1252,1707,21,1.768487,1.817372,-0.048886
383,Lachie Neale,Brisbane Lions,30.181778,28.977176,1.204602,1621,2182,25,1.383216,1.328010,0.055206
432,Marcus Bontempelli,Western Bulldogs,29.697070,30.626676,-0.929606,1158,1863,22,1.594046,1.643944,-0.049898


In [38]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,36.980404,39.051748,-2.071344,514,1601,19,2.309832,2.439210,-0.129378
599,Taylor Walker,Adelaide,29.404716,30.686255,-1.281540,553,1566,18,1.877696,1.959531,-0.081835
514,Patrick Cripps,Carlton,30.188070,31.022549,-0.834478,1252,1707,21,1.768487,1.817372,-0.048886
104,Charlie Curnow,Carlton,34.080586,36.425688,-2.345102,574,2022,22,1.685489,1.801468,-0.115979
432,Marcus Bontempelli,Western Bulldogs,29.697070,30.626676,-0.929606,1158,1863,22,1.594046,1.643944,-0.049898
311,Jeremy Cameron,Geelong,34.581821,37.182107,-2.600286,833,2214,24,1.561961,1.679409,-0.117447
591,Stephen Coniglio,Greater Western Sydney,27.430622,26.467992,0.962630,1104,1768,21,1.551506,1.497058,0.054447
38,Bayley Fritsch,Melbourne,31.053790,32.931280,-1.877489,514,2009,24,1.545734,1.639188,-0.093454
211,Isaac Heeney,Sydney,32.448135,33.466696,-1.018562,896,2136,25,1.519107,1.566793,-0.047685
97,Chad Warner,Sydney,29.269645,30.738944,-1.469298,1208,1938,24,1.510302,1.586117,-0.075815


In [39]:
player_ratings_2022.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,36.980404,39.051748,-2.071344,514,1601,19,2.309832,2.439210,-0.129378
311,Jeremy Cameron,Geelong,34.581821,37.182107,-2.600286,833,2214,24,1.561961,1.679409,-0.117447
104,Charlie Curnow,Carlton,34.080586,36.425688,-2.345102,574,2022,22,1.685489,1.801468,-0.115979
623,Tom Hawkins,Geelong,31.616207,33.756631,-2.140424,696,2366,25,1.336272,1.426738,-0.090466
211,Isaac Heeney,Sydney,32.448135,33.466696,-1.018562,896,2136,25,1.519107,1.566793,-0.047685
521,Peter Wright,Essendon,30.675617,33.417901,-2.742284,586,2162,22,1.418854,1.545694,-0.126840
38,Bayley Fritsch,Melbourne,31.053790,32.931280,-1.877489,514,2009,24,1.545734,1.639188,-0.093454
582,Shai Bolton,Richmond,28.434097,31.102253,-2.668156,903,1985,23,1.432448,1.566864,-0.134416
514,Patrick Cripps,Carlton,30.188070,31.022549,-0.834478,1252,1707,21,1.768487,1.817372,-0.048886
97,Chad Warner,Sydney,29.269645,30.738944,-1.469298,1208,1938,24,1.510302,1.586117,-0.075815


In [40]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,36.980404,39.051748,-2.071344,514,1601,19,2.309832,2.439210,-0.129378
599,Taylor Walker,Adelaide,29.404716,30.686255,-1.281540,553,1566,18,1.877696,1.959531,-0.081835
514,Patrick Cripps,Carlton,30.188070,31.022549,-0.834478,1252,1707,21,1.768487,1.817372,-0.048886
104,Charlie Curnow,Carlton,34.080586,36.425688,-2.345102,574,2022,22,1.685489,1.801468,-0.115979
311,Jeremy Cameron,Geelong,34.581821,37.182107,-2.600286,833,2214,24,1.561961,1.679409,-0.117447
432,Marcus Bontempelli,Western Bulldogs,29.697070,30.626676,-0.929606,1158,1863,22,1.594046,1.643944,-0.049898
38,Bayley Fritsch,Melbourne,31.053790,32.931280,-1.877489,514,2009,24,1.545734,1.639188,-0.093454
97,Chad Warner,Sydney,29.269645,30.738944,-1.469298,1208,1938,24,1.510302,1.586117,-0.075815
469,Mitch Lewis,Hawthorn,20.376140,21.876840,-1.500700,379,1382,15,1.474395,1.582984,-0.108589
638,Touk Miller,Gold Coast,27.606166,30.039462,-2.433296,1320,1907,22,1.447623,1.575221,-0.127598


In [41]:
player_ratings_2022.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
144,Darcy Moore,Collingwood,27.307212,9.662126,17.645086,1044,2314,24,1.180087,0.417551,0.762536
592,Steven May,Melbourne,25.017692,8.978103,16.039588,1106,2049,22,1.220971,0.438170,0.782801
286,James Sicily,Hawthorn,26.957935,12.856460,14.101476,1363,2110,22,1.277627,0.609311,0.668316
573,Sam Taylor,Greater Western Sydney,18.230144,5.072816,13.157327,1018,2132,22,0.855072,0.237937,0.617135
331,Jordan Ridley,Essendon,19.400047,6.491332,12.908715,968,1794,20,1.081385,0.361836,0.719549
73,Brennan Cox,Fremantle,19.046227,7.581338,11.464889,1024,2000,23,0.952311,0.379067,0.573244
188,Harris Andrews,Brisbane Lions,17.545272,6.343645,11.201627,948,2337,24,0.750760,0.271444,0.479317
555,Sam Collins,Gold Coast,17.863748,6.943103,10.920645,798,2154,22,0.829329,0.322335,0.506994
424,Luke Ryan,Fremantle,18.771796,8.056507,10.715288,1256,2040,24,0.920186,0.394927,0.525259
615,Tom Barrass,West Coast,17.075430,6.492941,10.582489,865,1767,19,0.966351,0.367456,0.598896


In [42]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
592,Steven May,Melbourne,25.017692,8.978103,16.039588,1106,2049,22,1.220971,0.438170,0.782801
144,Darcy Moore,Collingwood,27.307212,9.662126,17.645086,1044,2314,24,1.180087,0.417551,0.762536
331,Jordan Ridley,Essendon,19.400047,6.491332,12.908715,968,1794,20,1.081385,0.361836,0.719549
286,James Sicily,Hawthorn,26.957935,12.856460,14.101476,1363,2110,22,1.277627,0.609311,0.668316
573,Sam Taylor,Greater Western Sydney,18.230144,5.072816,13.157327,1018,2132,22,0.855072,0.237937,0.617135
615,Tom Barrass,West Coast,17.075430,6.492941,10.582489,865,1767,19,0.966351,0.367456,0.598896
617,Tom Clurey,Port Adelaide,10.513978,3.561098,6.952880,392,1182,13,0.889507,0.301277,0.588230
636,Tom Stewart,Geelong,21.953072,12.414779,9.538292,1173,1662,20,1.320883,0.746978,0.573904
73,Brennan Cox,Fremantle,19.046227,7.581338,11.464889,1024,2000,23,0.952311,0.379067,0.573244
162,Dylan Grimes,Richmond,10.194749,3.368911,6.825839,481,1228,15,0.830191,0.274341,0.555850


2023

In [43]:
player_ratings_2023.sort_values('vaep_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
105,Charlie Curnow,Carlton,45.305286,47.792261,-2.486975,699,1774.0,19.0,2.553849,2.694040,-0.140190
585,Taylor Walker,Adelaide,40.914770,43.801084,-2.886314,598,1473.0,18.0,2.777649,2.973597,-0.195948
424,Marcus Bontempelli,Western Bulldogs,38.409470,37.419880,0.989591,1418,1694.0,20.0,2.267383,2.208966,0.058417
486,Nick Larkey,North Melbourne,38.033913,40.429362,-2.395449,496,1728.0,18.0,2.201037,2.339662,-0.138626
490,Noah Anderson,Gold Coast,31.749414,30.095589,1.653825,1352,1607.0,19.0,1.975695,1.872781,0.102914
660,Zak Butters,Port Adelaide,31.388648,30.906492,0.482155,1410,1495.0,19.0,2.099575,2.067324,0.032251
573,Shai Bolton,Richmond,30.127189,30.223890,-0.096701,1107,1645.0,19.0,1.831440,1.837319,-0.005878
596,Toby Greene,Greater Western Sydney,28.644537,30.718942,-2.074405,787,1462.0,16.0,1.959271,2.101159,-0.141888
504,Oscar Allen,West Coast,28.371204,29.795311,-1.424107,543,1696.0,19.0,1.672830,1.756799,-0.083969
132,Dan Houston,Port Adelaide,28.242913,24.086345,4.156568,1248,1551.0,19.0,1.820949,1.552956,0.267993


In [44]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
585,Taylor Walker,Adelaide,40.914770,43.801084,-2.886314,598,1473.0,18.0,2.777649,2.973597,-0.195948
105,Charlie Curnow,Carlton,45.305286,47.792261,-2.486975,699,1774.0,19.0,2.553849,2.694040,-0.140190
424,Marcus Bontempelli,Western Bulldogs,38.409470,37.419880,0.989591,1418,1694.0,20.0,2.267383,2.208966,0.058417
486,Nick Larkey,North Melbourne,38.033913,40.429362,-2.395449,496,1728.0,18.0,2.201037,2.339662,-0.138626
660,Zak Butters,Port Adelaide,31.388648,30.906492,0.482155,1410,1495.0,19.0,2.099575,2.067324,0.032251
490,Noah Anderson,Gold Coast,31.749414,30.095589,1.653825,1352,1607.0,19.0,1.975695,1.872781,0.102914
98,Chad Warner,Sydney,24.167122,24.634554,-0.467433,1083,1227.0,15.0,1.969611,2.007706,-0.038096
596,Toby Greene,Greater Western Sydney,28.644537,30.718942,-2.074405,787,1462.0,16.0,1.959271,2.101159,-0.141888
102,Charlie Cameron,Brisbane Lions,27.643758,29.050125,-1.406367,496,1458.0,18.0,1.896005,1.992464,-0.096459
249,Jack Sinclair,St Kilda,27.973163,22.870983,5.102180,1478,1498.0,18.0,1.867367,1.526768,0.340599


In [45]:
player_ratings_2023.sort_values('offensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
105,Charlie Curnow,Carlton,45.305286,47.792261,-2.486975,699,1774.0,19.0,2.553849,2.694040,-0.140190
585,Taylor Walker,Adelaide,40.914770,43.801084,-2.886314,598,1473.0,18.0,2.777649,2.973597,-0.195948
486,Nick Larkey,North Melbourne,38.033913,40.429362,-2.395449,496,1728.0,18.0,2.201037,2.339662,-0.138626
424,Marcus Bontempelli,Western Bulldogs,38.409470,37.419880,0.989591,1418,1694.0,20.0,2.267383,2.208966,0.058417
660,Zak Butters,Port Adelaide,31.388648,30.906492,0.482155,1410,1495.0,19.0,2.099575,2.067324,0.032251
596,Toby Greene,Greater Western Sydney,28.644537,30.718942,-2.074405,787,1462.0,16.0,1.959271,2.101159,-0.141888
573,Shai Bolton,Richmond,30.127189,30.223890,-0.096701,1107,1645.0,19.0,1.831440,1.837319,-0.005878
490,Noah Anderson,Gold Coast,31.749414,30.095589,1.653825,1352,1607.0,19.0,1.975695,1.872781,0.102914
504,Oscar Allen,West Coast,28.371204,29.795311,-1.424107,543,1696.0,19.0,1.672830,1.756799,-0.083969
102,Charlie Cameron,Brisbane Lions,27.643758,29.050125,-1.406367,496,1458.0,18.0,1.896005,1.992464,-0.096459


In [46]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
585,Taylor Walker,Adelaide,40.914770,43.801084,-2.886314,598,1473.0,18.0,2.777649,2.973597,-0.195948
105,Charlie Curnow,Carlton,45.305286,47.792261,-2.486975,699,1774.0,19.0,2.553849,2.694040,-0.140190
486,Nick Larkey,North Melbourne,38.033913,40.429362,-2.395449,496,1728.0,18.0,2.201037,2.339662,-0.138626
424,Marcus Bontempelli,Western Bulldogs,38.409470,37.419880,0.989591,1418,1694.0,20.0,2.267383,2.208966,0.058417
596,Toby Greene,Greater Western Sydney,28.644537,30.718942,-2.074405,787,1462.0,16.0,1.959271,2.101159,-0.141888
660,Zak Butters,Port Adelaide,31.388648,30.906492,0.482155,1410,1495.0,19.0,2.099575,2.067324,0.032251
98,Chad Warner,Sydney,24.167122,24.634554,-0.467433,1083,1227.0,15.0,1.969611,2.007706,-0.038096
102,Charlie Cameron,Brisbane Lions,27.643758,29.050125,-1.406367,496,1458.0,18.0,1.896005,1.992464,-0.096459
407,Luke Breust,Hawthorn,21.968678,23.136432,-1.167754,487,1229.0,16.0,1.787525,1.882541,-0.095017
490,Noah Anderson,Gold Coast,31.749414,30.095589,1.653825,1352,1607.0,19.0,1.975695,1.872781,0.102914


In [47]:
player_ratings_2023.sort_values('defensive_value', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
147,Darcy Moore,Collingwood,22.922899,8.572295,14.350604,1003,1693.0,18.0,1.353981,0.506338,0.847643
226,Jack Buckley,Greater Western Sydney,23.373112,9.107561,14.265551,925,1681.0,18.0,1.390429,0.541794,0.848635
18,Aliir Aliir,Port Adelaide,20.319519,6.547307,13.772212,830,1855.0,19.0,1.095392,0.352955,0.742437
193,Harris Andrews,Brisbane Lions,23.800831,10.303752,13.497079,1031,1756.0,18.0,1.355400,0.586774,0.768626
417,Luke Ryan,Fremantle,21.640262,9.418455,12.221807,1305,1633.0,19.0,1.325184,0.576758,0.748427
487,Nick Murray,Adelaide,17.056918,5.129571,11.927347,520,1353.0,17.0,1.260674,0.379126,0.881548
546,Sam Collins,Gold Coast,18.506692,6.828052,11.678640,893,1855.0,19.0,0.997665,0.368089,0.629576
562,Sam Taylor,Greater Western Sydney,14.894492,3.687988,11.206504,639,1003.0,11.0,1.484994,0.367696,1.117298
101,Charlie Ballard,Gold Coast,19.651927,8.909995,10.741932,861,1815.0,19.0,1.082751,0.490909,0.591842
480,Nick Blakey,Sydney,21.758774,11.356765,10.402009,1267,1415.0,18.0,1.537723,0.802598,0.735124


In [48]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,vaep_value,offensive_value,defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
562,Sam Taylor,Greater Western Sydney,14.894492,3.687988,11.206504,639,1003.0,11.0,1.484994,0.367696,1.117298
487,Nick Murray,Adelaide,17.056918,5.129571,11.927347,520,1353.0,17.0,1.260674,0.379126,0.881548
226,Jack Buckley,Greater Western Sydney,23.373112,9.107561,14.265551,925,1681.0,18.0,1.390429,0.541794,0.848635
147,Darcy Moore,Collingwood,22.922899,8.572295,14.350604,1003,1693.0,18.0,1.353981,0.506338,0.847643
193,Harris Andrews,Brisbane Lions,23.800831,10.303752,13.497079,1031,1756.0,18.0,1.355400,0.586774,0.768626
417,Luke Ryan,Fremantle,21.640262,9.418455,12.221807,1305,1633.0,19.0,1.325184,0.576758,0.748427
18,Aliir Aliir,Port Adelaide,20.319519,6.547307,13.772212,830,1855.0,19.0,1.095392,0.352955,0.742437
480,Nick Blakey,Sydney,21.758774,11.356765,10.402009,1267,1415.0,18.0,1.537723,0.802598,0.735124
603,Tom Barrass,West Coast,13.756048,3.960107,9.795941,572,1341.0,14.0,1.025805,0.295310,0.730495
73,Brennan Cox,Fremantle,18.020721,8.262804,9.757917,937,1442.0,16.0,1.249703,0.573010,0.676693


In [49]:
action_vaep = get_vaep_action_summary(vaep_2021)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-104.618695,-67.117907,-37.500788,972,0.002816,-0.107632,-0.069051,-0.038581,-0.017732,-0.012532,-0.068912
5,Handball,-516.933187,-250.480428,-266.452758,60799,0.176132,-0.008502,-0.004120,-0.004383,-0.087615,-0.046767,-0.489641
12,Tackle,-0.696689,0.951814,-1.648503,129,0.000374,-0.005401,0.007378,-0.012779,-0.000118,0.000178,-0.003029
7,Kick,317.147953,618.962092,-301.814139,80643,0.233619,0.003933,0.007675,-0.003743,0.053753,0.115566,-0.554622
0,Carry,859.819226,839.175849,20.643377,54634,0.158272,0.015738,0.015360,0.000378,0.145730,0.156683,0.037935
11,Spoil,249.650289,-34.166573,283.816862,13590,0.039370,0.018370,-0.002514,0.020884,0.042313,-0.006379,0.521549
4,Gather,273.055358,250.151459,22.903899,14029,0.040641,0.019464,0.017831,0.001633,0.046280,0.046706,0.042089
6,Hard Ball Get,428.833392,259.978873,168.854519,18947,0.054889,0.022633,0.013721,0.008912,0.072683,0.048541,0.310292
13,Uncontested Mark,1184.559787,1155.645084,28.914703,38669,0.112022,0.030633,0.029886,0.000748,0.200770,0.215771,0.053134
10,Shot,323.089550,408.019824,-84.930275,9834,0.028489,0.032854,0.041491,-0.008636,0.054760,0.076181,-0.156070


In [50]:
action_vaep = get_vaep_action_summary(vaep_2022)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-129.013497,-82.104428,-46.909069,1198,0.003503,-0.107691,-0.068535,-0.039156,-0.021212,-0.014730,-0.092315
12,Tackle,-2.097799,-0.380267,-1.717531,84,0.000246,-0.024974,-0.004527,-0.020447,-0.000345,-0.000068,-0.003380
5,Handball,-524.277784,-244.528978,-279.748807,59270,0.173288,-0.008846,-0.004126,-0.004720,-0.086202,-0.043871,-0.550531
7,Kick,288.085678,606.389260,-318.303582,79965,0.233794,0.003603,0.007583,-0.003981,0.047367,0.108792,-0.626405
0,Carry,824.922309,799.677607,25.244702,53272,0.155752,0.015485,0.015011,0.000474,0.135634,0.143470,0.049680
11,Spoil,266.682994,-29.055783,295.738777,13964,0.040827,0.019098,-0.002081,0.021179,0.043848,-0.005213,0.581999
4,Gather,294.216805,265.490061,28.726744,13390,0.039148,0.021973,0.019827,0.002145,0.048375,0.047631,0.056533
6,Hard Ball Get,401.064853,252.513229,148.551624,17178,0.050223,0.023348,0.014700,0.008648,0.065943,0.045303,0.292342
13,Uncontested Mark,1217.105341,1185.157535,31.947806,38101,0.111396,0.031944,0.031106,0.000839,0.200117,0.212629,0.062872
9,Loose Ball Get,1245.714321,926.003927,319.710395,37170,0.108674,0.033514,0.024913,0.008601,0.204820,0.166134,0.629174


In [51]:
action_vaep = get_vaep_action_summary(vaep_2023)
action_vaep.sort_values("vaep_value_per_action")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-110.285434,-68.540164,-41.745269,1068,0.003096,-0.103264,-0.064176,-0.039087,-0.018143,-0.012285,-0.083562
5,Handball,-526.263966,-226.054135,-300.209830,61695,0.178862,-0.008530,-0.003664,-0.004866,-0.086575,-0.040518,-0.600936
12,Tackle,-0.805908,0.560023,-1.365931,101,0.000293,-0.007979,0.005545,-0.013524,-0.000133,0.000100,-0.002734
7,Kick,272.233796,609.354293,-337.120497,79022,0.229095,0.003445,0.007711,-0.004266,0.044785,0.109220,-0.674821
0,Carry,843.294837,802.160750,41.134087,55393,0.160592,0.015224,0.014481,0.000743,0.138729,0.143778,0.082339
11,Spoil,261.498412,-33.322032,294.820444,13665,0.039617,0.019136,-0.002438,0.021575,0.043019,-0.005973,0.590148
4,Gather,295.250572,267.464253,27.786320,13573,0.039350,0.021753,0.019706,0.002047,0.048571,0.047940,0.055620
6,Hard Ball Get,391.328498,244.980021,146.348477,16277,0.047189,0.024042,0.015051,0.008991,0.064377,0.043910,0.292949
13,Uncontested Mark,1298.275054,1259.098681,39.176374,38100,0.110457,0.034075,0.033047,0.001028,0.213577,0.225679,0.078420
9,Loose Ball Get,1338.672466,985.907774,352.764693,38727,0.112275,0.034567,0.025458,0.009109,0.220223,0.176713,0.706137


In [52]:
player = "Nick Daicos"
player_data = get_player(vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-4.193291,-1.467071,-2.726220,283,0.198457,-0.014817,-0.005184,-0.009633,-0.298086,-0.091997,1.450517
1,Error,-0.943311,-0.208755,-0.734557,6,0.004208,-0.157219,-0.034792,-0.122426,-0.067057,-0.013091,0.390829
10,Spoil,-0.105424,-0.077274,-0.028151,5,0.003506,-0.021085,-0.015455,-0.005630,-0.007494,-0.004846,0.014978
7,Knock On,0.071713,0.032653,0.039060,4,0.002805,0.017928,0.008163,0.009765,0.005098,0.002048,-0.020782
3,Gather,0.542152,0.540675,0.001477,44,0.030856,0.012322,0.012288,0.000034,0.038540,0.033905,-0.000786
5,Hard Ball Get,0.846489,0.535367,0.311122,47,0.032959,0.018010,0.011391,0.006620,0.060174,0.033572,-0.165536
9,Shot,1.159600,1.347990,-0.188390,17,0.011921,0.068212,0.079294,-0.011082,0.082432,0.084530,0.100235
6,Kick,1.469955,3.162401,-1.692446,344,0.241234,0.004273,0.009193,-0.004920,0.104494,0.198308,0.900485
2,Free For,2.307627,1.418924,0.888704,28,0.019635,0.082415,0.050676,0.031739,0.164041,0.088978,-0.472845
11,Uncontested Mark,2.630174,2.554462,0.075713,124,0.086957,0.021211,0.020600,0.000611,0.186969,0.160186,-0.040284


In [53]:
player = "Nick Daicos"
player_data = get_player(vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-2.945189,-1.145184,-1.800005,300,0.226929,-0.009817,-0.003817,-0.006000,-0.118757,-0.049877,-0.978130
1,Error,-0.130735,-0.096722,-0.034013,3,0.002269,-0.043578,-0.032241,-0.011338,-0.005272,-0.004213,-0.018483
10,Spoil,-0.051860,-0.015409,-0.036451,3,0.002269,-0.017287,-0.005136,-0.012150,-0.002091,-0.000671,-0.019808
7,Knock On,0.129387,0.102994,0.026393,2,0.001513,0.064694,0.051497,0.013197,0.005217,0.004486,0.014342
5,Hard Ball Get,0.346807,0.162924,0.183883,30,0.022693,0.011560,0.005431,0.006129,0.013984,0.007096,0.099923
9,Shot,1.615457,2.065886,-0.450430,39,0.029501,0.041422,0.052971,-0.011549,0.065139,0.089978,-0.244765
3,Gather,1.946410,1.402758,0.543652,59,0.044629,0.032990,0.023776,0.009214,0.078484,0.061096,0.295423
11,Uncontested Mark,2.591204,2.624944,-0.033740,70,0.052950,0.037017,0.037499,-0.000482,0.104483,0.114327,-0.018334
2,Free For,2.874253,2.346314,0.527939,24,0.018154,0.119761,0.097763,0.021997,0.115896,0.102192,0.286884
6,Kick,3.107472,3.758243,-0.650771,286,0.216339,0.010865,0.013141,-0.002275,0.125300,0.163687,-0.353632


In [54]:
player = "Christian Petracca"
player_data = get_player(vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
10,Shot,-2.908194,-2.582645,-0.325549,69,0.048729,-0.042148,-0.037430,-0.004718,-0.126951,-0.115715,-0.552670
5,Handball,-1.260050,-0.308544,-0.951506,316,0.223164,-0.003987,-0.000976,-0.003011,-0.055005,-0.013824,-1.615329
2,Error,-0.950634,-0.893933,-0.056701,6,0.004237,-0.158439,-0.148989,-0.009450,-0.041498,-0.040052,-0.096259
11,Spoil,-0.324264,-0.338980,0.014716,5,0.003531,-0.064853,-0.067796,0.002943,-0.014155,-0.015188,0.024982
1,Contested Mark,0.074287,0.071651,0.002636,4,0.002825,0.018572,0.017913,0.000659,0.003243,0.003210,0.004476
8,Knock On,0.088882,-0.016401,0.105282,6,0.004237,0.014814,-0.002733,0.017547,0.003880,-0.000735,0.178733
3,Free For,1.991134,1.857385,0.133749,16,0.011299,0.124446,0.116087,0.008359,0.086918,0.083220,0.227060
7,Kick,2.280213,2.840478,-0.560264,288,0.203390,0.007917,0.009863,-0.001945,0.099537,0.127267,-0.951135
12,Uncontested Mark,3.442492,3.397126,0.045366,103,0.072740,0.033422,0.032982,0.000440,0.150274,0.152208,0.077016
6,Hard Ball Get,3.573125,2.854596,0.718529,102,0.072034,0.035031,0.027986,0.007044,0.155977,0.127900,1.219815


In [55]:
player = "Christian Petracca"
player_data = get_player(vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-1.686826,-0.906809,-0.780017,321,0.233115,-0.005255,-0.002825,-0.002430,-0.062712,-0.033596,8.296420
10,Shot,-1.601631,-1.182437,-0.419194,79,0.057371,-0.020274,-0.014968,-0.005306,-0.059545,-0.043807,4.458634
2,Error,-1.062711,-0.828852,-0.233859,12,0.008715,-0.088559,-0.069071,-0.019488,-0.039509,-0.030707,2.487376
11,Spoil,0.058138,0.061554,-0.003415,6,0.004357,0.009690,0.010259,-0.000569,0.002161,0.002280,0.036327
8,Knock On,0.178416,0.130817,0.047599,2,0.001452,0.089208,0.065409,0.023799,0.006633,0.004847,-0.506271
1,Contested Mark,1.127557,1.178089,-0.050532,6,0.004357,0.187926,0.196348,-0.008422,0.041920,0.043646,0.537467
6,Hard Ball Get,1.904234,1.471286,0.432949,106,0.076979,0.017964,0.013880,0.004084,0.070795,0.054508,-4.604930
3,Free For,2.403995,2.071077,0.332918,20,0.014524,0.120200,0.103554,0.016646,0.089375,0.076730,-3.540988
7,Kick,2.882206,3.498098,-0.615892,242,0.175744,0.011910,0.014455,-0.002545,0.107154,0.129598,6.550748
0,Carry,2.981504,3.296668,-0.315165,194,0.140886,0.015369,0.016993,-0.001625,0.110845,0.122135,3.352155


In [56]:
def get_team(vaep_data, team):
    
    return vaep_data[vaep_data['team'] == team]

In [57]:
team_data = get_team(vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-37.983910,-22.195144,-15.788765,3861,0.181310,-0.009838,-0.005749,-0.004089,-0.091976,-0.059204,-0.414597
2,Error,-9.264556,-7.231631,-2.032926,76,0.003569,-0.121902,-0.095153,-0.026749,-0.022434,-0.019290,-0.053383
12,Tackle,-0.389170,-0.094573,-0.294597,3,0.000141,-0.129723,-0.031524,-0.098199,-0.000942,-0.000252,-0.007736
8,Knock On,10.543484,8.632545,1.910940,221,0.010378,0.047708,0.039061,0.008647,0.025531,0.023027,0.050179
11,Spoil,14.427491,-2.525682,16.953174,762,0.035783,0.018934,-0.003315,0.022248,0.034936,-0.006737,0.445173
4,Gather,20.330404,18.057372,2.273032,861,0.040432,0.023613,0.020973,0.002640,0.049229,0.048167,0.059687
7,Kick,23.495685,34.316239,-10.820553,4833,0.226955,0.004862,0.007100,-0.002239,0.056894,0.091536,-0.284137
6,Hard Ball Get,27.872517,17.903582,9.968935,1042,0.048932,0.026749,0.017182,0.009567,0.067492,0.047757,0.261774
1,Contested Mark,37.702545,28.983456,8.719090,387,0.018173,0.097423,0.074893,0.022530,0.091295,0.077311,0.228954
10,Shot,38.685665,45.004397,-6.318732,708,0.033247,0.054641,0.063566,-0.008925,0.093676,0.120046,-0.165923


In [58]:
team_data = get_team(vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-29.253274,-10.309574,-18.943700,3101,0.170873,-0.009433,-0.003325,-0.006109,-0.085542,-0.032360,-0.810111
2,Error,-7.042057,-5.581319,-1.460738,64,0.003527,-0.110032,-0.087208,-0.022824,-0.020592,-0.017519,-0.062467
12,Tackle,0.112281,0.097252,0.015029,5,0.000276,0.022456,0.019450,0.003006,0.000328,0.000305,0.000643
7,Kick,6.856246,26.521582,-19.665335,4276,0.235618,0.001603,0.006202,-0.004599,0.020049,0.083246,-0.840971
8,Knock On,7.252108,6.051755,1.200353,135,0.007439,0.053719,0.044828,0.008892,0.021206,0.018995,0.051332
4,Gather,15.159719,13.397970,1.761749,665,0.036643,0.022797,0.020147,0.002649,0.044330,0.042054,0.075340
11,Spoil,17.022904,-0.100502,17.123406,668,0.036808,0.025483,-0.000150,0.025634,0.049778,-0.000315,0.732268
6,Hard Ball Get,25.353957,18.390393,6.963564,853,0.047002,0.029723,0.021560,0.008164,0.074139,0.057724,0.297791
10,Shot,29.575581,35.086618,-5.511037,597,0.032896,0.049540,0.058772,-0.009231,0.086484,0.110130,-0.235675
1,Contested Mark,35.890236,29.233241,6.656994,373,0.020553,0.096220,0.078373,0.017847,0.104949,0.091757,0.284681


In [59]:
team_data = get_team(vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-30.338179,-13.402347,-16.935833,3556,0.184430,-0.008532,-0.003769,-0.004763,-0.081538,-0.037867,-0.933793
2,Error,-7.585577,-5.133196,-2.452381,69,0.003579,-0.109936,-0.074394,-0.035542,-0.020387,-0.014503,-0.135217
12,Tackle,-0.411268,-0.008129,-0.403139,3,0.000156,-0.137089,-0.002710,-0.134380,-0.001105,-0.000023,-0.022228
8,Knock On,9.619745,7.042869,2.576875,273,0.014159,0.035237,0.025798,0.009439,0.025855,0.019899,0.142081
7,Kick,10.694203,32.406803,-21.712600,4332,0.224677,0.002469,0.007481,-0.005012,0.028742,0.091561,-1.197170
11,Spoil,11.505390,-0.948634,12.454024,712,0.036928,0.016159,-0.001332,0.017492,0.030922,-0.002680,0.686679
6,Hard Ball Get,15.622135,9.732617,5.889517,753,0.039054,0.020747,0.012925,0.007821,0.041987,0.027498,0.324731
4,Gather,16.912346,14.607423,2.304924,796,0.041284,0.021247,0.018351,0.002896,0.045454,0.041271,0.127087
1,Contested Mark,39.519522,29.645601,9.873920,361,0.018723,0.109472,0.082121,0.027352,0.106215,0.083760,0.544419
10,Shot,40.803859,47.297719,-6.493860,582,0.030185,0.070110,0.081268,-0.011158,0.109667,0.133634,-0.358053


In [60]:
team_data = get_team(vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("vaep_value")

,action_type,vaep_value,offensive_value,defensive_value,num_actions,action_%,vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-30.720041,-14.558053,-16.161988,3670,0.179471,-0.008371,-0.003967,-0.004404,-0.098231,-0.051523,-0.535574
2,Error,-7.176687,-4.252124,-2.924563,77,0.003765,-0.093204,-0.055222,-0.037981,-0.022948,-0.015049,-0.096914
12,Tackle,0.399038,0.178984,0.220054,4,0.000196,0.099759,0.044746,0.055014,0.001276,0.000633,0.007292
7,Kick,7.192054,25.592336,-18.400282,4837,0.236540,0.001487,0.005291,-0.003804,0.022998,0.090575,-0.609746
8,Knock On,7.867301,6.157230,1.710071,199,0.009732,0.039534,0.030941,0.008593,0.025157,0.021791,0.056668
10,Shot,11.419100,15.993062,-4.573962,530,0.025918,0.021545,0.030176,-0.008630,0.036514,0.056602,-0.151571
4,Gather,11.653893,11.404192,0.249700,760,0.037166,0.015334,0.015006,0.000329,0.037265,0.040361,0.008275
11,Spoil,11.840206,-0.912841,12.753047,701,0.034280,0.016890,-0.001302,0.018193,0.037861,-0.003231,0.422609
6,Hard Ball Get,23.350226,12.565175,10.785051,946,0.046261,0.024683,0.013282,0.011401,0.074665,0.044470,0.357394
1,Contested Mark,34.994934,25.562117,9.432817,350,0.017116,0.099986,0.073035,0.026951,0.111901,0.090468,0.312583
